# Output Ovewview

In [38]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import seaborn as sns
from pathlib import Path
from IPython.display import display
import pickle
# from env_utils import read_pso_search_files, detect_convergence
%matplotlib inline

from data_utils import find_match_from_seach_collector, get_unique_search_df, get_time_diff_df

In [39]:
def get_collision_rate(exp_path):
    search_files = [f for f in os.listdir(exp_path) if f.startswith('search_') and f.endswith('.csv')]
    search_df = pd.read_csv(f"{exp_path}/{search_files[0]}")

    ttc_zero_count = 0
    for index, row in search_df.iterrows():
        if row['ttc'] == 0:
            ttc_zero_count += 1
    return ttc_zero_count / len(search_df)

def get_collision_rate_species(exp_path):
    # if "species.pkl" not in os.listdir(exp_path), return 0
    if "species.pkl" not in os.listdir(exp_path):
        return 0
    with open(exp_path / "species.pkl", "rb") as f:
        species = pickle.load(f)
    search_files = [f for f in os.listdir(exp_path) if f.startswith('search_') and f.endswith('.csv')]
    search_df = pd.read_csv(f"{exp_path}/{search_files[0]}")
    final_test_set = []
    ttcs = []
    for s in species:
        for p in s:
            if p.best is not None:
                match_row = find_match_from_seach_collector(p.best, search_df)
                ttc = match_row['ttc']
                ttcs.append(ttc)
    # count how many rows are there which has ttc=0
    ttc_zero_count = 0
    for ttc in ttcs:
        if ttc == 0:
            ttc_zero_count += 1
    # print(f"ttc_zero_count: {ttc_zero_count}, total: {len(ttcs)}, ratio: {ttc_zero_count / len(ttcs)}")
    if len(ttcs) == 0:
        return 0
    return ttc_zero_count / len(ttcs)

def get_acd_unique(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    # count how many rows are there which has ttc=0
    ttc_zero_count = 0
    total_distance = 0
    for index, row in search_df_unique.iterrows():
        total_distance += row['distance']
        if row['ttc'] == 0:
            ttc_zero_count += 1
    if ttc_zero_count == 0:
        return total_distance
    # KM
    return total_distance / ttc_zero_count /1000

def get_acd_last20(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    search_df_last20 = search_df_unique.tail(20)
    ttc_zero_count = 0
    total_distance = 0
    for index, row in search_df_last20.iterrows():
        total_distance += row['distance']
        if row['ttc'] == 0:
            ttc_zero_count += 1
    if ttc_zero_count == 0:
        return total_distance
    # KM
    return total_distance / ttc_zero_count /1000

def get_collision_rate_unique(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    # count how many rows are there which has ttc=0
    ttc_zero_count = 0
    for index, row in search_df_unique.iterrows():
        if row['ttc'] == 0:
            ttc_zero_count += 1
    return ttc_zero_count / len(search_df_unique)

def get_act_unique(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    search_df_unique = get_time_diff_df(search_df_unique)
    ttc_zero_count = 0
    total_time = 0
    for index, row in search_df_unique.iterrows():
        total_time += row['time_consumed']
        if row['ttc'] == 0:
            ttc_zero_count += 1
    if ttc_zero_count == 0:
        return total_time
    # sec
    # print(f"total_time: {total_time}, ttc_zero_count: {ttc_zero_count}")
    return total_time / ttc_zero_count

def get_act_last20(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    search_df_last20 = search_df_unique.tail(20)
    search_df_last20 = get_time_diff_df(search_df_last20)
    ttc_zero_count = 0
    total_time = 0
    for index, row in search_df_last20.iterrows():
        total_time += row['time_consumed']
        if row['ttc'] == 0:
            ttc_zero_count += 1
    if ttc_zero_count == 0:
        return total_time
    # sec
    return total_time / ttc_zero_count

def get_average_ttc_unique(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    ttc_sum = 0
    for index, row in search_df_unique.iterrows():
        if row['ttc'] >= 10:
            ttc_sum += 10
        else:
            ttc_sum += row['ttc']
    return ttc_sum / len(search_df_unique)

def get_average_ttc_last20(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    search_df_last20 = search_df_unique.tail(20)
    ttc_sum = 0
    for index, row in search_df_last20.iterrows():
        if row['ttc'] >= 10:
            ttc_sum += 10
        else:
            ttc_sum += row['ttc']
    return ttc_sum / len(search_df_last20)

def get_collision_rate_last20(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    search_df_last20 = search_df_unique.tail(20)
    # count how many rows are there which has ttc=0
    ttc_zero_count = 0
    for index, row in search_df_last20.iterrows():
        if row['ttc'] == 0:
            ttc_zero_count += 1
    return ttc_zero_count / len(search_df_last20)

def to_percent(x):
    return round(x*100, 2)

In [40]:
agents_data = {}

base_path = Path("../../output-final")
# base_path = Path("../../DataPack/output-BehAVExplor")

# ADS_name = ["apollo","ba","interfuser","tfpp","highway"]
ADS_name = ["apollo"]

for scenario_name in os.listdir(base_path):
    scenario_path = os.path.join(base_path, scenario_name)
    if os.path.isdir(scenario_path):
        for agent_name in os.listdir(scenario_path):

            # Optinal: select one agent
            if agent_name not in ADS_name:
                continue
            agent_path = os.path.join(scenario_path, agent_name)
            if os.path.isdir(agent_path):
                if agent_name not in agents_data:
                    agents_data[agent_name] = []
                for experiment_name in os.listdir(agent_path):

                    if "20x50" not in experiment_name:
                        continue

                    # if experiment_name.split("_")[1] is a float, then it is a weight
                    if experiment_name.split("_")[1].replace(".", "", 1).isdigit():
                        w = float(experiment_name.split("_")[1])
                    else:
                        w = 0
                

                    exp_path = os.path.join(agent_path, experiment_name)
                    collision_rate = get_collision_rate(Path(exp_path))
                    collision_rate_species = get_collision_rate_species(Path(exp_path))
                    collision_rate_unique = get_collision_rate_unique(Path(exp_path))
                    collision_rate_last20 = get_collision_rate_last20(Path(exp_path))
                    acd_unique = get_acd_unique(Path(exp_path))
                    acd_last20 = get_acd_last20(Path(exp_path))
                    act_unique = get_act_unique(Path(exp_path))
                    act_last20 = get_act_last20(Path(exp_path))
                    average_ttc_unique = get_average_ttc_unique(Path(exp_path))
                    average_ttc_last20 = get_average_ttc_last20(Path(exp_path))

                    # Add the scenario name and experiment name to the list of the agent
                    agents_data[agent_name].append(
                        (
                            scenario_name,
                            experiment_name,
                            w,
                            to_percent(collision_rate),
                            to_percent(collision_rate_species),
                            to_percent(collision_rate_unique),
                            to_percent(collision_rate_last20),
                            acd_unique,
                            acd_last20,
                            act_unique,
                            act_last20,
                            average_ttc_unique,
                            average_ttc_last20
                        )
                    )

# Create a DataFrame for each agent
agents_dfs = {}
metrics = ["CR", "CR-S", "CR-U", "CR-L20", "ACD-U", "ACD-L20", "ACT-U", "ACT-L20", "ATTC-U", "ATTC-L20"]
for agent_name, experiments in agents_data.items():
    # Create a DataFrame
    df = pd.DataFrame(
        experiments,
        columns=["Scenario", "Experiment", "w"] + metrics,
    )
    df = df.groupby("Scenario", group_keys=False).apply(lambda x: x.sort_values("w"))

    df.drop("Experiment", axis=1, inplace=True)
    agents_dfs[agent_name] = df

for ads in ADS_name:
    print(ads)
    # if ads == "apollo":
    #     display(agents_dfs[ads])
    # display(agents_dfs[ads])

apollo


In [41]:
for ads in ADS_name:
    print(f"======{ads}======")
    agent_df_copy = agents_dfs[ads].copy()
    # drop scenario column
    agent_df_copy.drop("Scenario", axis=1, inplace=True)
    grouped_df = agent_df_copy.groupby("w").mean()
    grouped_df = grouped_df.round(2)
    display(grouped_df)

    # Calculate the mean of the metrics
    mean_df = grouped_df.mean()
    # round the values
    mean_df = mean_df.round(2)
    display(mean_df["CR-U"])


    # fig, axs = plt.subplots(math.ceil(len(metrics) // 2), 2, figsize=(20, 26))
    # fig.suptitle(f"{ads} - Grouped by w", fontsize=16)

    # for metric in metrics:
    #     grouped_df[metric].plot(ax=axs.flatten()[metrics.index(metric)], title=metric)

    # plt.show()

    # convert here: https://www.convertcsv.com/html-table-to-csv.htm

======apollo======


,CR,CR-S,CR-U,CR-L20,ACD-U,ACD-L20,ACT-U,ACT-L20,ATTC-U,ATTC-L20
w,,,,,,,,,,
0.0,3.85,2.63,4.00,7.50,24.46,1125.56,9911.12,1165.82,5.61,5.45
0.3,21.56,0.00,20.85,22.50,6.39,1749.19,3653.42,1367.70,3.63,2.90
0.5,26.38,0.00,25.46,13.33,1.40,1539.20,1193.81,1392.26,3.17,3.24
0.7,33.65,13.64,32.83,30.00,0.93,609.56,483.72,436.81,3.13,2.55
1.0,37.72,0.00,37.89,36.67,0.48,457.52,356.97,551.86,2.48,2.92


24.21

In [42]:
scenario_name_list = ["front_brake", "front_cut_in_with_one_npc", "front_cut_in_with_two_npc",
                      "opposite_vehicle_taking_priority",
                      "nonsignalized_junction_left_turn",
                      "nonsignalized_junction_right_turn"]

path_name = str(base_path).split("/")[-1]

# get all w values
w_list = []
for ads in ADS_name:
    w_list += agents_dfs[ads]["w"].unique().tolist()
w_list = list(set(w_list))
# order w_list by descending order
w_list.sort(reverse=True)

# Construct a new list, which contains w, ads_name, CR-U under each scenario
summary_list = []

for w in w_list:
    for ads in ADS_name:
        row = [w, ads]
        for scenario_name in scenario_name_list:
            cr_u = agents_dfs[ads][(agents_dfs[ads]["Scenario"] == scenario_name) & (agents_dfs[ads]["w"] == w)]["CR-U"].values
            if len(cr_u) == 0:
                cr_u = None
            else:
                cr_u = cr_u[0]
            row.append(cr_u)
        summary_list.append(row)

summary_df = pd.DataFrame(summary_list, columns=["w", "ADS"] + scenario_name_list)

# group by w
grouped_summary_df = summary_df.groupby("w")
# in each group, sort the ADS by the order in ADS_name
grouped_summary_df = grouped_summary_df.apply(lambda x: x.set_index("ADS").reindex(ADS_name).reset_index())



display(summary_df)
# save to csv
summary_df.to_csv(f"summary_cru_{path_name}.csv", index=False)

,w,ADS,front_brake,front_cut_in_with_one_npc,front_cut_in_with_two_npc,opposite_vehicle_taking_priority,nonsignalized_junction_left_turn,nonsignalized_junction_right_turn
0,1.0,apollo,50.40,15.10,14.8,44.7,46.2,56.14
1,0.7,apollo,47.95,13.30,5.5,37.9,44.1,48.20
2,0.5,apollo,34.60,8.95,3.8,19.0,39.9,46.50
3,0.3,apollo,28.20,1.10,1.0,16.6,35.4,42.80
4,0.0,apollo,10.70,0.20,0.5,2.2,3.2,7.20
